In [ ]:
# accelerate를 이용하여 sdxl 1.0 모델에 dreambooth 기법이 적용된 LoRA를 생성해 학습시키고 저장한다.
# 실행환경 : Colab L4 or A100
# BPP LoRA 및 다양한 스타일 LoRA를 데이터셋 기반으로 학습 가능

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/huggingface/diffusers # accelerate를 사용하기 위해 패키지 clone

In [ ]:
os.chdir('/content/diffusers')

In [ ]:
!pip install huggingface_hub datasets # 요구사항 및 필요 패키지 설치
!pip install -e .

In [ ]:
os.chdir('/content/diffusers/examples/dreambooth')
!pip install -r requirements_sdxl.txt
!pip install --upgrade peft

In [ ]:
# 허깅페이스 로그인
!huggingface-cli login

In [ ]:
# wandb 로그인 (사용하려면 해당 셀을 실행한 뒤 accelerate argument로 --report_to="wandb"를 추가한다.)
!wandb login

In [ ]:
#accelerate 설정 초기화
!accelerate config default

In [ ]:
# cli에서 사용할 임시 변수 설정
os.environ["DATASET"] = "" # 원하는 허깅페이스 데이터셋 경로 지정하거나 내용 조절해서 새로 만든 데이터셋
os.environ["OUPUT_DIR"] = "" # 저장할 경로(임의로 지정, 허깅페이스 로그인시 아아디에 같은 이름으로 repository 생성됨)
os.environ["VAE"] = "madebyollin/sdxl-vae-fp16-fix" # 기본 모델의 vae보다 일반적으로 성능이 좋은 vae
os.environ["BASE_MODEL"] = "stabilityai/stable-diffusion-xl-base-1.0" # sdxl base 1.0 모델

In [ ]:
# 학습을 가속하기 위해 accelerate 사용
# *** 주요 arguments ***
# dataset_name or instance_image_dir : 학습에 사용할 이미지
# --image_column : 데이터셋에 이미지가 있는 컬럼
# --instance_prompt : 매 이미지마다 학습할 프롬프트.
# --caption_column : 데이터셋에 매 이미지마다 다르게 넣어줄 캡션에대한 컬럼이 있을시, 사용할 수 있음
# --validation_prompt : 설정시 추가 공간을 요구하지만 학습을 더 빠르게 시킬 수 있음
# --class_prompt, --with_prior_preservation : dreambooth로 이미지를 특정 토큰에 학습시킬 때,
#                                             학습에 사용한 이미지가 침범하지 않길 바라는 class가 있다면
#                                             해당하는 클래스에 대한 이미지와 클래스의 프롬프트, 그리고 강도(--prior_loss_weight)를 설정할 수 있다.
# --train_text_encoder : 텍스트 인코더 또한 학습할지 결정
# --resolution : 학습에 사용될 이미지가 resize될 크기 (default : 1024 (1024*1024로 설정된다.))
# --learnig_rate : 학습 계수
# --max_train_steps : 최대 학습 횟수, learning_rate와 함께 학습에 가장 중요하다.

!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path=$BASE_MODEL \
  --dataset_name=$DATASET \
  --instance_prompt="" \
  --pretrained_vae_model_name_or_path=$VAE \
  --output_dir=$OUPUT_DIR \
  --mixed_precision="fp16" \
  --caption_column="text" \
  --resolution=1024 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --checkpointing_steps=500 \
  --max_train_steps=1000 \
  --validation_epochs=25 \
  --seed="0" \
  --push_to_hub